In [ ]:
import numpy
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as im

In [ ]:
import os
folder = r"D:\part3"
files = os.listdir(folder)
files

In [ ]:
fd = cv2.CascadeClassifier(  r"C:\Users\h00236409\kaizen_training\kaizen\haarcascade_frontalface_alt.xml"  )
def get_face(path):
    img=cv2.imread(path)
    corners = fd.detectMultiScale(img,1.3,4)
    if len(corners)==0:
        return None,None 
    else:
        (x,y,w,h)= corners[0]
        img = img[x:x+h,y:y+w]   # cropping the image and resize them,
        img = cv2.resize(img,(100,100))
        return (x,y,w,h),img

In [ ]:
trainimg = []
trainlb_age = []
trainlb_gender = []
trainlb_race = []
for file in files:
    if ".jpg" in file:
        path = folder+'\\'+file 
        details = file.split("_")
        age = details[0]
        gender = details[1]
        race = details[2]
        if int(age)>18 and int(age)<50:
            try:
                corner,img = get_face(path)
                if corner !=None:
                    trainimg.append(img)
                    trainlb_age.append(int(age))
                    trainlb_gender.append(int(gender))
                    trainlb_race.append(race)
            except :
                print("failed")

In [ ]:
trainimg = numpy.array(trainimg)
trainlb_age = numpy.array(trainlb_age).reshape(1893,1) 
trainlb_gender = numpy.array(trainlb_gender).reshape(1893,1) 
trainlb_race = numpy.array(trainlb_race)

trianimg = trainimg/255

#reshape the image data 
trainimg = trainimg.reshape(1893,100,100,3)  

from sklearn.preprocessing import OneHotEncoder
trainlb_race = OneHotEncoder().fit_transform(trainlb_race.reshape(1893,1)).toarray() # should excute once والا بيطلع ايرور

print(trainimg.shape)
print(trainlb_age.shape)
print(trainlb_gender.shape)
print(trainlb_race.shape)




In [ ]:
from sklearn.metrics import mean_squared_error


# NN_Age

In [ ]:
# features extraction for images
# model 
# building the model
from keras import models, layers 
model = models.Sequential()

# add first convolutional and maxpoling layer
model.add(layers.Conv2D(filters=20,kernel_size=(3,3),activation='relu',input_shape=(100,100,3))) # بدهاش تزبط يا زلمه
model.add(layers.MaxPooling2D(pool_size=(2,2)))

# add the second convolutional layer
model.add(layers.Conv2D(filters=40,kernel_size=(3,3),activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

# add the flatten layer
model.add(layers.Flatten())
model.add(layers.Dense(20,activation='relu'))
model.add(layers.Dense(1))

In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [ ]:
model.fit(trianimg,trainlb_age,epochs=20,batch_size=50,shuffle =True, verbose = True)

In [ ]:
model.save("agemodel.h5")

# NN_Gender

In [ ]:
print (trainlb_gender)

In [ ]:
# building the model
from keras import models, layers 
model2 = models.Sequential()

# add first convolutional and maxpoling layer
model2.add(layers.Conv2D(filters=20,kernel_size=(3,3),activation='relu',input_shape=(100,100,3)))
model2.add(layers.MaxPooling2D(pool_size=(2,2)))

# add the second convolutional layer
model2.add(layers.Conv2D(filters=40,kernel_size=(3,3),activation='relu'))
model2.add(layers.MaxPooling2D(pool_size=(2,2)))

# add the flatten layer
model2.add(layers.Flatten())
model2.add(layers.Dense(20,activation='relu'))
model2.add(layers.Dense(1,activation= 'sigmoid')) # how many nurons

In [ ]:
model2.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['accuracy'])

In [ ]:
model2.fit(trianimg,trainlb_gender,epochs=10,batch_size=50,shuffle =True, verbose = True)

In [ ]:
model2.save("gendermodel.h5")

# NN_Race

In [ ]:
print(trainimg.shape)
print(trainlb_race.shape)

In [ ]:
# building the model
from keras import models, layers 
model3 = models.Sequential()

# add first convolutional and maxpoling layer
model3.add(layers.Conv2D(filters=20,kernel_size=(3,3),activation='relu',input_shape=(100,100,3)))
model3.add(layers.MaxPooling2D(pool_size=(2,2)))

# add the second convolutional layer
model3.add(layers.Conv2D(filters=40,kernel_size=(3,3),activation='relu'))
model3.add(layers.MaxPooling2D(pool_size=(2,2)))

# add the flatten layer
model3.add(layers.Flatten())
model3.add(layers.Dense(20,activation='relu'))
model3.add(layers.Dense(5,activation= 'softmax'))

In [ ]:
model3.compile(optimizer='adam',loss='categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
model3.fit(trianimg,trainlb_race,epochs=10,batch_size=50,shuffle =True, verbose = True)

In [ ]:
model3.save("racemodel.h5")

In [ ]:
label1=['male','female']
label2=['White', 'Black', 'Asian', 'Indian','Hispanic']

In [ ]:
from keras import models

In [ ]:
model = models.load_model("agemodel.h5")

In [ ]:
model2 = models.load_model("gendermodel.h5")

In [ ]:
model3 = models.load_model("racemodel.h5")

In [ ]:
def get_face2(img):
    corners = fd.detectMultiScale(img,1.3,4)
    if len(corners)==0:
        return None,None 
    else:
        (x,y,w,h)= corners[0]
        img = img[x:x+h,y:y+w]   # cropping the image and resize them,
        img = cv2.resize(img,(100,100))
        return (x,y,w,h),img

In [ ]:
vid=cv2.VideoCapture(0)
while True:
    ret,img = vid.read()
    print(ret)
        #img2=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    corner,img2=get_face2(img)
    if corner !=None:
        age_out= model.predict(img2.reshape(1,100,100,3))
        gender_out= model2.predict_classes(img2.reshape(1,100,100,3))
        race_out= model3.predict_classes(img2.reshape(1,100,100,3))
        #emotion=labels[output[0]]
        text = str(age_out[0][0])+' '+label1[gender_out[0][0]]+' '+label2[race_out[0]]
        (x,y,w,h) = corner
        cv2.putText(img,text,(x,y),cv2.FONT_HERSHEY_COMPLEX,1.0,(0,0,255),2)
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
        cv2.imshow('img',img)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
vid.release()
cv2.destroyAllWindows()

In [ ]:
race_out[0]

In [ ]:
vid.release()

In [ ]:
model.summary()

In [ ]:
def get_cosmetics(dender,race)
    if gender=="male" and race=="white":
        return "product1"
    elif gender=="female" and race=="white":
        return "product2"
    if gender=="male" and race=="Black":
         return "product3"
    elif gender=="female" and race=="Black":
        return "product4"